# constraints
    you must give refrence on image (blue points) after that any 2 points will read it by this refrence 

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image_path = r"D:\Vortex\ROV\Phase 2\Tasks2025\task 6\image2.png"  # Adjust to your image path
image = cv2.imread(image_path)
if image is None:
    print("Image not found. Please check the path.")
    exit()

# Define the known reference length in cm
KNOWN_REFERENCE_LENGTH_CM = 36.0

# Placeholder for reference points and scale
ref_points = []
scale = None

# Mouse callback function to select points
def select_points(event, x, y, flags, param):
    global ref_points, scale
    
    # Store points on left mouse click
    if event == cv2.EVENT_LBUTTONDOWN:
        ref_points.append((x, y))
        cv2.circle(image, (x, y), 5, (255, 0, 0), -1)  # Mark the point with a small circle
        
        # Once two points are selected, calculate the scale
        if len(ref_points) == 2:
            # Calculate pixel distance between points
            point1, point2 = ref_points
            pixel_distance = np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)
            scale = KNOWN_REFERENCE_LENGTH_CM / pixel_distance  # cm per pixel
            print(f"Scale calculated: {scale:.4f} cm/pixel")

# Create a window and set the mouse callback
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", select_points)

# Show the image and let the user select the reference points
print("Please select two points to define a 36 cm reference length.")
while True:
    cv2.imshow("Image", image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q") or len(ref_points) == 2:  # Press 'q' to quit or after selecting 2 points
        break

cv2.destroyAllWindows()

# Verify that the scale has been calculated
if scale is None:
    print("Reference scale was not set. Please rerun and select two points.")
    exit()

# Allow user to measure additional distances manually
measure_points = []
image_copy = image.copy()

def measure_distances(event, x, y, flags, param):
    global measure_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        measure_points.append((x, y))
        cv2.circle(image_copy, (x, y), 5, (0, 255, 0), -1)
        
        # When two points are selected, calculate and display distance
        if len(measure_points) == 2:
            pt1, pt2 = measure_points
            pixel_distance = np.sqrt((pt2[0] - pt1[0]) ** 2 + (pt2[1] - pt1[1]) ** 2)
            real_distance = pixel_distance * scale  # Convert to cm
            cv2.line(image_copy, pt1, pt2, (0, 255, 0), 2)
            cv2.putText(image_copy, f"{real_distance:.2f} cm", 
                        ((pt1[0] + pt2[0]) // 2, (pt1[1] + pt2[1]) // 2 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            print(f"Measured distance: {real_distance:.2f} cm")
            measure_points.clear()  # Reset for additional measurements

# Set the callback function for manual measurements
cv2.namedWindow("Measure Distances")
cv2.setMouseCallback("Measure Distances", measure_distances)

# Display the image and wait for the user to measure distances
print("Select two points to measure the distance between them. Press 'q' to quit.")
while True:
    cv2.imshow("Measure Distances", image_copy)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cv2.destroyAllWindows()


Please select two points to define a 36 cm reference length.
Scale calculated: 0.3099 cm/pixel
Select two points to measure the distance between them. Press 'q' to quit.
Measured distance: 42.81 cm
Measured distance: 56.42 cm
Measured distance: 28.53 cm
Measured distance: 45.87 cm
Measured distance: 31.61 cm
